In [1]:
from dataclasses import dataclass
from pathlib import Path

import pandas as pd

from mmvae_hub.experiment_vis.utils import *
from mmvae_hub.base.utils.MongoDB import MongoDatabase
from mmvae_hub.base.utils.flags_utils import get_config_path, BaseFlagsSetup
from mmvae_hub.base.utils.utils import dict2pyobject, json2dict

[17:19:03 CEST] Log file: /usr/local/var/logs/mmvae_hub/UTC-20210524-151903.log


In [2]:
exp_db = MongoDatabase(training=False)
experiments = exp_db.connect()

In [3]:
print('There are currently ', experiments.count_documents({}), 'experiments in the database.')

There are currently  45 experiments in the database.


In [4]:
df = make_experiments_dataframe(experiments)

In [10]:
show_cols = ['weighted_mixture','score', 'method', 'end_epoch', 'class_dim', 'beta', '_id', 'score_lr', 'score_gen','score_prd', 'version', 'expvis_url']
df[show_cols].sort_values(by=['score'], ascending=False).head(10)

,weighted_mixture,score,method,end_epoch,class_dim,beta,_id,score_lr,score_gen,score_prd,version,expvis_url
2,0.0,17.147867,joint_elbo,619,512.0,2.5,polymnist_joint_elbo_2021_05_23_12_09_30_507370,5.471367,11.676500,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/6f29c649ff03.pdf
1,1.0,15.753367,planar_mixture_5,469,512.0,1.0,polymnist_planar_mixture_2021_05_23_12_10_02_9...,2.934167,12.819200,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/e2fa2c7c7da1.pdf
0,1.0,14.164517,planar_mixture_5,489,512.0,1.0,polymnist_planar_mixture_2021_05_23_12_10_03_0...,2.807067,11.357450,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/57259190dc8c.pdf
4,1.0,12.212250,planar_mixture_5,539,512.0,2.5,polymnist_planar_mixture_2021_05_23_12_10_31_0...,1.991600,10.220650,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/9735fb5d581b.pdf
3,1.0,11.179667,planar_mixture_5,459,512.0,2.5,polymnist_planar_mixture_2021_05_23_12_10_31_4...,1.832900,9.346767,0.0,0.0.5-dev,https://ppb.hendrikklug.xyz/2b23461ce868.pdf


# Explore all pfom experiments

In [13]:
df.loc[df['method'].str.startswith('pfom')][[*show_cols]].sort_values(by=['score'], ascending=False)

,weighted_mixture,score,method,end_epoch,class_dim,beta,_id,score_lr,score_gen,score_prd,version,expvis_url


In [16]:
df.loc[df['dir_experiment_run'].str.startswith('/cluster')][[*show_cols]].sort_values(by=['score'], ascending=False)

df.loc[df['_id'].str.startswith('polymnist_planar_mixture_2021_05_23_12_10_02_9')]._id.item()

'polymnist_planar_mixture_2021_05_23_12_10_02_979139'

# Generate samples with the model that has the highest score.

In [8]:
best_score_id = df[df.score == df.score.max()]._id.item()

print(best_score_id)
experiments_database = MongoDatabase(training=False, _id=best_score_id)
experiment_dict = experiments_database.get_experiment_dict()
config = json2dict(get_config_path())
experiment_flags = experiment_dict['flags']

flags = BaseFlagsSetup.set_paths_with_config(config, experiment_flags, is_dict=True)

experiment_flags['use_clf'] = False
experiment_flags['device'] = 'cpu'
experiment_flags['weighted_mixture'] = False
experiment_flags['save_figure'] = False



polymnist_joint_elbo_2021_05_23_12_09_30_507370


In [9]:
show_generated_figs(flags=dict2pyobject(experiment_flags))

AttributeError: can't set attribute